In [8]:
import numpy as np
from numpy import pi
import sympy as sp

from constants import *
from util import *

In [9]:
from IPython.core.display import display, HTML, Latex
display(HTML("<style>.container { width:100% !important; }</style>"))

In [10]:
C, f_c, f_a, Lambda_QCD, kappa, N, rho_c, g_0, g_osc, T_0, gamma, theta_i, eV = \
 sp.symbols("C, f_c, f_a, Lambda_QCD, kappa, N, rho_c, g_0, g_osc, T_0, gamma, theta_i, eV")
GeV = 10**9 * eV
T_osc = kappa * (10**16 * GeV / (f_a / N))**(sp.Rational(1, 6))
m_a = 6*10**(-10) * eV * 10**16 * GeV / (f_a / N)
zeta = C * (Lambda_QCD / T_osc)**4 # ok
m_a_osc = m_a * zeta # ok
# s up to a constant factor
s0 = g_0 * T_0**3 # ok
s_osc = g_osc * T_osc**3 # ok
n_a = f_c / 2 * m_a_osc * (f_a / N)**2 * theta_i**2 
rho_a = n_a * m_a * s0 / s_osc * gamma
Omega_a = rho_a / rho_c # ok
Omega_a 

18.0*10**(5/6)*C*Lambda_QCD**4*T_0**3*eV**4*f_c*g_0*gamma*theta_i**2/(g_osc*kappa**7*rho_c*(N*eV/f_a)**(7/6))

In [11]:
from constants import *
C = 0.018
g_star = 61.75
kappa = sp.RR(((10 * M_pl**2 / (pi**2 * g_star))**(1/4) * C**(1/2) * Lambda_QCD**2 * (6e-6 / 1e4)**(1/2))**(1/3))
g_star_s0 = 3.91
g_star_s_osc = g_star 
f_c = 1.44
iota = 18 * 10 ** (5 / 6) * C * Lambda_QCD**4 * T0**3 * f_c * g_star_s0 / (g_star_s_osc * kappa**7 * rho_c)
show("iota =", iota / h**2, ", kappa =", kappa)
iota * (1e16 * 1e9)**(7/6) / h**2

iota = 4.860644e-46 , kappa = 1.324125e+08 


7.13444903096115e-17